In [267]:
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit.quantum_info import SparsePauliOp, Pauli
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.units import DistanceUnit
import numpy as np
from qiskit_nature.second_q.mappers import JordanWignerMapper



In [268]:
import pennylane as qml
import numpy as np

import warnings
warnings.filterwarnings(action="ignore", category=np.ComplexWarning)

# Load the dataset
h2 = qml.data.load("qchem", molname="H2", bondlength=0.742, basis="STO-3G")[0]

# Extract the Hamiltonian
H, n_qubits = h2.hamiltonian, len(h2.hamiltonian.wires)
print(H)
from qiskit.quantum_info import SparsePauliOp

pauli_terms= [
    ("IIII", -0.09963387941370971),
    ("ZIII",  0.17110545123720233),
    ("IZII",  0.17110545123720225),
    ("ZZII",  0.16859349595532533),
    ("YXXY",  0.04533062254573469),
    ("YYXX", -0.04533062254573469),
    ("XXYY", -0.04533062254573469),
    ("XYYX",  0.04533062254573469),
    ("IIZI", -0.22250914236600539),
    ("ZIZI",  0.12051027989546245),
    ("IIIZ", -0.22250914236600539),
    ("ZIIZ",  0.16584090244119712),   # 此项疑似笔误，应为 ZII**Z**
    ("IZZI",  0.16584090244119712),
    ("IZIZ",  0.12051027989546245),
    ("IIZZ",  0.1743207725924201)
]

hamiltonian3= SparsePauliOp.from_list(pauli_terms)

# ✅ 打印结果


-0.09963387941370971 * I(0) + 0.17110545123720233 * Z(0) + 0.17110545123720225 * Z(1) + 0.16859349595532533 * (Z(0) @ Z(1)) + 0.04533062254573469 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.04533062254573469 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.04533062254573469 * (X(0) @ X(1) @ Y(2) @ Y(3)) + 0.04533062254573469 * (X(0) @ Y(1) @ Y(2) @ X(3)) + -0.22250914236600539 * Z(2) + 0.12051027989546245 * (Z(0) @ Z(2)) + -0.22250914236600539 * Z(3) + 0.16584090244119712 * (Z(0) @ Z(3)) + 0.16584090244119712 * (Z(1) @ Z(2)) + 0.12051027989546245 * (Z(1) @ Z(3)) + 0.1743207725924201 * (Z(2) @ Z(3))


In [269]:
#hamiltonian4 pennylane论坛

# Pauli 字符串与对应系数
pauli_terms = [
    ("IIII", -0.8105479805373279),
    ("IIIZ", 0.1721839326191554),
    ("IIZI", -0.22575349222402372),
    ("IZII", 0.17218393261915543),
    ("ZIII", -0.2257534922240237),
    ("IIZZ", 0.12091263261776627),
    ("IZIZ", 0.16892753870087907),
    ("YYYY", 0.045232799946057826),
    ("XXYY", 0.045232799946057826),
    ("YYXX", 0.045232799946057826),
    ("XXXX", 0.045232799946057826),
    ("ZIIZ", 0.1661454325638241),
    ("IZZI", 0.1661454325638241),
    ("ZIZI", 0.17464343068300453),
    ("ZZII", 0.12091263261776627),
]

# 构建 SparsePauliOp
hamiltonian4= SparsePauliOp.from_list(pauli_terms)

# ✅ 打印结果
print(hamiltonian4)



SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IZII', 'ZIII', 'IIZZ', 'IZIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'ZIIZ', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-0.81054798+0.j,  0.17218393+0.j, -0.22575349+0.j,  0.17218393+0.j,
 -0.22575349+0.j,  0.12091263+0.j,  0.16892754+0.j,  0.0452328 +0.j,
  0.0452328 +0.j,  0.0452328 +0.j,  0.0452328 +0.j,  0.16614543+0.j,
  0.16614543+0.j,  0.17464343+0.j,  0.12091263+0.j])


In [270]:
#Hamiltonian1 来自pyscf
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.742",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

molecule = driver.run()
mapper = JordanWignerMapper()
hamiltonian1 = mapper.map(molecule.hamiltonian.second_q_op())
print(hamiltonian1)

SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IIZZ', 'IZII', 'IZIZ', 'ZIII', 'ZIIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-0.81280876+0.j,  0.17110568+0.j, -0.22250985+0.j,  0.12051037+0.j,
  0.17110568+0.j,  0.16859357+0.j, -0.22250985+0.j,  0.16584097+0.j,
  0.0453306 +0.j,  0.0453306 +0.j,  0.0453306 +0.j,  0.0453306 +0.j,
  0.16584097+0.j,  0.17432084+0.j,  0.12051037+0.j])


In [271]:
#Hamiltonian2 来自文献
terms = [
    ("IIII", -0.807184),        # -0.807184 I⊗I⊗I⊗I
    ("ZIZI",  0.175106),        #  0.175106 Z⊗I⊗Z⊗I (q0和q2)
    ("IZIZ",  0.169404),        #  0.169404 I⊗Z⊗I⊗Z (q1和q3)
    ("IIZI", -0.230474),        # -0.230474 I⊗I⊗Z⊗I (q0)
    ("ZIII", -0.230474),        # -0.230474 Z⊗I⊗I⊗I (q3)
    ("IIIZ",  0.173740),        #  0.173740 I⊗I⊗I⊗Z (q3)
    ("IZII",  0.173740),        #  0.173740 I⊗Z⊗I⊗I (q2)
    ("YYYY",  0.045094),        #  0.045094 Y⊗Y⊗Y⊗Y (q0,q1,q2,q3)
    ("XXYY",  0.045094),        #  0.045094 X⊗X⊗Y⊗Y (q0,q1,q2,q3)
    ("YYXX",  0.045094),        #  0.045094 Y⊗Y⊗X⊗X (q0,q1,q2,q3)
    ("XXXX",  0.045094),        #  0.045094 X⊗X⊗X⊗X (q0,q1,q2,q3)
    ("XIIZ",  0.166582),        #  0.166582 X⊗I⊗I⊗Z (q0和q3)
    ("IZZI",  0.166582),        #  0.166582 I⊗Z⊗Z⊗I (q1和q2)
    ("ZZII",  0.121488),        #  0.121488 Z⊗Z⊗I⊗I (q2和q3)
    ("IIZZ",  0.121488)         #  0.121488 I⊗I⊗Z⊗Z (q0和q1)
]

# 轉換為 SparsePauliOp（自動合併同類項）
hamiltonian2 = SparsePauliOp.from_list(
    [(pauli, coeff + 0.j) for pauli, coeff in terms]  # 係數需為複數
).simplify()

print("Qiskit 哈密頓量:")
print(hamiltonian2)

Qiskit 哈密頓量:
SparsePauliOp(['IIII', 'ZIZI', 'IZIZ', 'IIZI', 'ZIII', 'IIIZ', 'IZII', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'XIIZ', 'IZZI', 'ZZII', 'IIZZ'],
              coeffs=[-0.807184+0.j,  0.175106+0.j,  0.169404+0.j, -0.230474+0.j, -0.230474+0.j,
  0.17374 +0.j,  0.17374 +0.j,  0.045094+0.j,  0.045094+0.j,  0.045094+0.j,
  0.045094+0.j,  0.166582+0.j,  0.166582+0.j,  0.121488+0.j,  0.121488+0.j])


In [272]:

cost_h=hamiltonian1

driver_scaling = 0.27
driver_h = SparsePauliOp.from_list([
    ("XIII", -1.0*driver_scaling),
    ("IXII", -1.0*driver_scaling),
    ("IIXI", -1.0*driver_scaling),
    ("IIIX", -1.0*driver_scaling),
])
print(driver_h)

from qiskit.quantum_info import SparsePauliOp

def build_commutator(op_a: SparsePauliOp, op_b: SparsePauliOp) -> SparsePauliOp:
    ab = op_a @ op_b
    ba = op_b @ op_a
    comm_pre= ab - ba
    comm = comm_pre*1j
    return comm.simplify()

comm_h=build_commutator(driver_h, cost_h)
print(comm_h)
print(build_commutator(driver_h, driver_h))

SparsePauliOp(['XIII', 'IXII', 'IIXI', 'IIIX'],
              coeffs=[-0.27+0.j, -0.27+0.j, -0.27+0.j, -0.27+0.j])
SparsePauliOp(['YIII', 'YIIZ', 'ZYYY', 'ZYXX', 'YIZI', 'YZII', 'IYII', 'IYIZ', 'YZYY', 'YZXX', 'IYZI', 'ZYII', 'IIYI', 'IIYZ', 'YYZY', 'XXZY', 'IZYI', 'ZIYI', 'IIIY', 'IIZY', 'IZIY', 'ZIIY', 'YYYZ', 'XXYZ'],
              coeffs=[ 0.12015532+0.j, -0.08955412+0.j,  0.02447852+0.j,  0.02447852+0.j,
 -0.09413326+0.j, -0.0650756 +0.j, -0.09239707+0.j, -0.09104053+0.j,
  0.02447852+0.j,  0.02447852+0.j, -0.08955412+0.j, -0.0650756 +0.j,
  0.12015532+0.j, -0.0650756 +0.j,  0.02447852+0.j,  0.02447852+0.j,
 -0.08955412+0.j, -0.09413326+0.j, -0.09239707+0.j, -0.0650756 +0.j,
 -0.09104053+0.j, -0.08955412+0.j,  0.02447852+0.j,  0.02447852+0.j])
SparsePauliOp(['IIII'],
              coeffs=[0.+0.j])


In [273]:
from qiskit.circuit import QuantumCircuit


# ----------- Pauli Gate 演化 -------------
def apply_pauli_evolution(qc, pauli_str, angle):
    n = qc.num_qubits
    pauli_str = pauli_str[::-1]  # Qiskit 是 little-endian
    indices = [i for i, p in enumerate(pauli_str) if p != 'I']
    if not indices:
        return

    for i in indices:
        if pauli_str[i] == 'X':
            qc.h(i)
        elif pauli_str[i] == 'Y':
            qc.sdg(i)
            qc.h(i)

    if len(indices) > 1:
        for i in range(len(indices) - 1):
            qc.cx(indices[i], indices[i + 1])
        target = indices[-1]
    else:
        target = indices[0]

    qc.rz(2 * angle, target)

    if len(indices) > 1:
        for i in reversed(range(len(indices) - 1)):
            qc.cx(indices[i], indices[i + 1])

    for i in indices:
        if pauli_str[i] == 'X':
            qc.h(i)
        elif pauli_str[i] == 'Y':
            qc.h(i)
            qc.s(i)


# ----------- 单层 FALQON 结构 -------------
def apply_falqon_layer(qc, beta_k, cost_h, driver_h, delta_t):

    for label, coeff in zip(cost_h.paulis.to_labels(), cost_h.coeffs):
        angle = float(coeff.real * delta_t)
        apply_pauli_evolution(qc, label, angle)

    # 1. 先 driver 演化（状态相关）
    for label, coeff in zip(driver_h.paulis.to_labels(), driver_h.coeffs):
        angle = float(coeff.real * beta_k * delta_t)
        apply_pauli_evolution(qc, label, angle)

    # 2. 再 cost 演化（状态无关）



from qiskit import transpile


def falqon_qiskit_fast_final_gate_count(n_qubits, n_layers, beta_1, delta_t, cost_h, driver_h, comm_h, 
                                        ):
    beta = [beta_1]
    energies = []

    state = Statevector.from_label("+" * n_qubits)
    final_circuit = QuantumCircuit(n_qubits)
    final_circuit.h(range(n_qubits))  # 初始 H^{⊗n}
    
    for i in range(n_layers):
        qc_layer = QuantumCircuit(n_qubits)
        apply_falqon_layer(qc_layer, beta[i], cost_h, driver_h, delta_t)
        final_circuit.compose(qc_layer, inplace=True)


        state = state.evolve(qc_layer)

        next_beta = -1*state.expectation_value(comm_h).real
        beta.append(next_beta)

        energy = state.expectation_value(cost_h).real
        

        energies.append(energy)
        

        print(f"第{i + 1}步：能量 = {energy:.8f} Ha | β_{i} = {beta[i]:+.6f}")
                # ✅ 判断是否达到误差容限
        if i >= 1:
            if abs(energies[-1] - energies[-2]) < 1e-6:
                print(f"✅ 收敛于第 {i + 1} 层，能量变化小于 1e-6 Ha")
                break


        # ✅ 判断是否达到误差容限
     

    return beta, energies, final_circuit


from qiskit import transpile

# 基础门集合（例如 IBM 超导平台）

# 执行主算法
basis = ["ecr", "id", "rz", "sx", "x"]
basis2= ['rz', 'rx', 'cx']  # IBM 超导平台的典型基础门集
n_layers = 1000
beta_1 = 1
delta_t = 0.03
res_beta, res_energies, final_circuit = falqon_qiskit_fast_final_gate_count(
    n_qubits=4,
    n_layers=n_layers,
    beta_1=beta_1,
    delta_t=delta_t,
    cost_h=cost_h,
    driver_h=driver_h,
    comm_h=comm_h
)
#print(final_circuit.draw())

from qiskit.transpiler import CouplingMap
coupling_map = CouplingMap.from_full(4)
from qiskit.transpiler import CouplingMap

# 定义线性拓扑：0–1–2–3
linear_coupling = CouplingMap(couplinglist=[(0, 1), (1, 0), (1, 2), (2, 1), (2, 3), (3, 2)])
transpiled_circuit= transpile(
    final_circuit,
    basis_gates=basis2,  # 典型 IBM 超导平台的基础门集合
    coupling_map=linear_coupling,  # 线性拓扑
    optimization_level=3
)

for gate, count in transpiled_circuit.count_ops().items():
    print(f"{gate}: {count}")

# ✅ 最后统一 transpile 并统计基础门数量
#transpiled_circuit = transpile(final_circuit, basis_gates=basis, optimization_level=1)
final_gate_counts = transpiled_circuit.count_ops()

# ✅ 打印统计
print("\n📊 最终电路在指定基础门集下的门数量统计：")
for gate, count in final_gate_counts.items():
    print(f"{gate}: {count}")



第1步：能量 = -0.76785714 Ha | β_0 = +1.000000
第2步：能量 = -0.76784282 Ha | β_1 = +0.013471
第3步：能量 = -0.76785036 Ha | β_2 = +0.026932
第4步：能量 = -0.76789058 Ha | β_3 = +0.040368
第5步：能量 = -0.76797419 Ha | β_4 = +0.053765
第6步：能量 = -0.76811170 Ha | β_5 = +0.067111
第7步：能量 = -0.76831344 Ha | β_6 = +0.080390
第8步：能量 = -0.76858947 Ha | β_7 = +0.093591
第9步：能量 = -0.76894955 Ha | β_8 = +0.106698
第10步：能量 = -0.76940310 Ha | β_9 = +0.119699
第11步：能量 = -0.76995915 Ha | β_10 = +0.132577
第12步：能量 = -0.77062631 Ha | β_11 = +0.145320
第13步：能量 = -0.77141271 Ha | β_12 = +0.157912
第14步：能量 = -0.77232597 Ha | β_13 = +0.170339
第15步：能量 = -0.77337319 Ha | β_14 = +0.182587
第16步：能量 = -0.77456085 Ha | β_15 = +0.194641
第17步：能量 = -0.77589485 Ha | β_16 = +0.206487
第18步：能量 = -0.77738042 Ha | β_17 = +0.218111
第19步：能量 = -0.77902212 Ha | β_18 = +0.229497
第20步：能量 = -0.78082383 Ha | β_19 = +0.240634
第21步：能量 = -0.78278869 Ha | β_20 = +0.251506
第22步：能量 = -0.78491914 Ha | β_21 = +0.262101
第23步：能量 = -0.78721683 Ha | β_22 = +0.272406
第24步：能量